In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

print('Loading data ...')

train = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\train.csv")
prop = pd.read_csv("final_prop_encoded_onehot.csv")

print('done')

Loading data ...
done


In [2]:
prop.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,garagecarcnt,...,propertylandusetypeid_264.0,propertylandusetypeid_265.0,propertylandusetypeid_266.0,propertylandusetypeid_267.0,propertylandusetypeid_269.0,propertylandusetypeid_270.0,propertylandusetypeid_275.0,propertylandusetypeid_279.0,propertylandusetypeid_31.0,propertylandusetypeid_47.0
0,10754147,0.0,0.0,5.0,3652.0,3652.0,6037.0,0.0,5.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,10759547,0.0,0.0,1.0,800.0,800.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10843547,0.0,0.0,1.0,73026.0,1274.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,10859147,0.0,0.0,2.0,5068.0,1041.0,6037.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,10879947,0.0,0.0,1.0,1776.0,784.0,6037.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
# print(prop.shape)
# prop['airconditioningtypeid'] = prop['airconditioningtypeid'].astype(str)
# prop['buildingqualitytypeid'] = prop['buildingqualitytypeid'].astype(str)
# prop['heatingorsystemtypeid'] = prop['heatingorsystemtypeid'].astype(str)
# prop['pooltypeid7'] = prop['pooltypeid7'].astype(str)
# prop['propertylandusetypeid'] = prop['propertylandusetypeid'].astype(str)
# cat = ['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid','pooltypeid7','propertylandusetypeid']
# prefixes = {
#     'airconditioningtypeid':'airconditioningtypeid',
#     'buildingqualitytypeid':'buildingqualitytypeid',
#     'heatingorsystemtypeid':'heatingorsystemtypeid',
#     'pooltypeid7':'pooltypeid7',
#     'propertylandusetypeid':'propertylandusetypeid'
# }
# prop = pd.get_dummies(prop, prefix=prefixes, columns=cat)
# print(prop.shape)

In [4]:
train.head()

,parcelid,logerror,transactiondate,subdate
0,11016594,0.0276,1/1/2016,201601
1,14366692,-0.1684,1/1/2016,201601
2,12098116,-0.0040,1/1/2016,201601
3,12643413,0.0218,2/1/2016,201601
4,14432541,-0.0050,2/1/2016,201601


In [5]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'subdate'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 110000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

print("Prepare for the prediction ...")
sample = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\sample_submission.csv")
sample.columns = ['parcelid','201704','201705','201706','201707','201708','201709']
df_test = sample.merge(prop, on='parcelid', how='left')
del prop; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

print("Start write result ...")

for c in sample.columns[sample.columns != 'parcelid']:
    sample[c] = p_test

submission = sample
del sample

(113039, 70) (113039,)
[1]	valid_0's l1: 0.0731014
[2]	valid_0's l1: 0.0731004
[3]	valid_0's l1: 0.0730986
[4]	valid_0's l1: 0.073098
[5]	valid_0's l1: 0.0730973
[6]	valid_0's l1: 0.0730952
[7]	valid_0's l1: 0.0730944
[8]	valid_0's l1: 0.0730931
[9]	valid_0's l1: 0.0730923
[10]	valid_0's l1: 0.0730906
[11]	valid_0's l1: 0.0730908
[12]	valid_0's l1: 0.0730903
[13]	valid_0's l1: 0.0730887
[14]	valid_0's l1: 0.0730869
[15]	valid_0's l1: 0.0730857
[16]	valid_0's l1: 0.0730847
[17]	valid_0's l1: 0.0730835
[18]	valid_0's l1: 0.0730828
[19]	valid_0's l1: 0.073082
[20]	valid_0's l1: 0.0730819
[21]	valid_0's l1: 0.0730812
[22]	valid_0's l1: 0.0730799
[23]	valid_0's l1: 0.073078
[24]	valid_0's l1: 0.0730769
[25]	valid_0's l1: 0.0730761
[26]	valid_0's l1: 0.0730748
[27]	valid_0's l1: 0.0730741
[28]	valid_0's l1: 0.0730733
[29]	valid_0's l1: 0.0730714
[30]	valid_0's l1: 0.07307
[31]	valid_0's l1: 0.0730681
[32]	valid_0's l1: 0.0730664
[33]	valid_0's l1: 0.0730659
[34]	valid_0's l1: 0.0730643
[35]	

[293]	valid_0's l1: 0.0729524
[294]	valid_0's l1: 0.0729525
[295]	valid_0's l1: 0.0729524
[296]	valid_0's l1: 0.0729519
[297]	valid_0's l1: 0.0729516
[298]	valid_0's l1: 0.072952
[299]	valid_0's l1: 0.0729512
[300]	valid_0's l1: 0.0729511
[301]	valid_0's l1: 0.0729513
[302]	valid_0's l1: 0.0729513
[303]	valid_0's l1: 0.0729509
[304]	valid_0's l1: 0.0729508
[305]	valid_0's l1: 0.0729503
[306]	valid_0's l1: 0.0729503
[307]	valid_0's l1: 0.0729498
[308]	valid_0's l1: 0.0729496
[309]	valid_0's l1: 0.0729499
[310]	valid_0's l1: 0.0729498
[311]	valid_0's l1: 0.0729502
[312]	valid_0's l1: 0.0729497
[313]	valid_0's l1: 0.0729493
[314]	valid_0's l1: 0.0729494
[315]	valid_0's l1: 0.0729494
[316]	valid_0's l1: 0.0729496
[317]	valid_0's l1: 0.0729498
[318]	valid_0's l1: 0.0729499
[319]	valid_0's l1: 0.0729498
[320]	valid_0's l1: 0.0729498
[321]	valid_0's l1: 0.0729496
[322]	valid_0's l1: 0.0729493
[323]	valid_0's l1: 0.0729497
[324]	valid_0's l1: 0.07295
[325]	valid_0's l1: 0.0729508
[326]	valid_0

In [8]:
from math import fabs as absolute
test_path = "C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\test.csv"
test = pd.read_csv(test_path, low_memory=False)
test_size=len(test)

In [9]:
total_ae = 0.0
print("Processing ", test_size, "rows")
for index,row in test.iterrows():
    r = submission.loc[submission['parcelid'] == row[0]]
    
    pred_error = r[str(row[3])]
    total_ae += absolute(pred_error - row[1])
    
    if index % 10000 == 0 and index !=0:
        print("Completed: ", index, "rows")
        
print("===Process Completed===")
print("\n Final score:", total_ae/test_size)

Processing  54849 rows
Completed:  10000 rows
Completed:  20000 rows
Completed:  30000 rows
Completed:  40000 rows
Completed:  50000 rows
===Process Completed===

 Final score: 0.0691565326383013


7